In [1]:
import json 
import requests
from IPython.display import clear_output
import pandas as pd
from pandas import json_normalize
import numpy as np
from datetime import datetime, date, timedelta
from tqdm import tqdm, trange
import warnings
warnings.filterwarnings("ignore")

In [2]:
#@title Константные значения
#@markdown За сколько дней осуществляется парсинг
N =  25#@param {type:"integer"}
#@markdown Текущий адрес API hh.ru
url = 'https://api.hh.ru/vacancies' #@param {type:"string"}

In [3]:
df = pd.DataFrame() # Новый дата фрейм
for dayLeft in range(N-1, -1, -1): # цикл обхода дней
    data=[]
    startDate = datetime.strftime(datetime.now() - timedelta(days=N), '%Y-%m-%d') # Дата начала парсинга
    for i in trange(1,20): #Ограничение страниц (2к записей, 19 стр)
        print('\nОбрабатывается дата: ' + (str)(startDate) + '\nОсталось: ' + (str)(dayLeft) + '\\' + (str)(N))
        parametrs = {'specialization': '1','per_page':'100', 'page':i, "date_from":startDate, 'date_to':startDate} #параметры для API hh.ru
        jsonResponce = requests.get(url, params=parametrs).json()
        if str(jsonResponce).startswith("{'items': [], 'found"): # если на странице уже нет рез-тов, тогда дропаем цикл
            break
        data.append(jsonResponce)
        vacancy_details = data[0]['items'][0].keys()
        df_temp = pd.DataFrame() #дата фрейм дня
        ind = 0
        for a in range(len(data)):
            try:
                for j in range(len(data[a]['items'])):
                    df_temp.loc[ind, 'name'] = data[a]['items'][j]['name']
                    df_temp.loc[ind, 'area'] = data[a]['items'][j]['area']['name']
                    if data[a]['items'][j]['salary']:
                        df_temp.loc[ind, 'minSalary'] = data[a]['items'][j]['salary']['from']
                        df_temp.loc[ind, 'maxSalary'] = data[a]['items'][j]['salary']['to']
                    df_temp.loc[ind, 'employer'] = data[a]['items'][j]['employer']['name']
                    df_temp.loc[ind, 'published_at'] = data[a]['items'][j]['published_at']
                    df_temp.loc[ind, 'schedule'] = data[a]['items'][j]['schedule']['name']
                    df_temp.loc[ind, 'responsibility'] = data[a]['items'][j]['snippet']['responsibility']
                    df_temp.loc[ind, 'requirement'] = data[a]['items'][j]['snippet']['requirement']
                    ind+=1
            except Exception:
                print(Exception)
        clear_output() 
    df = pd.concat([df, df_temp], ignore_index=True)




100%|██████████| 19/19 [00:36<00:00,  1.91s/it]


In [4]:
#@markdown Сохраняем в csv файл
df.to_csv('hh_ru_'+startDate+'-'+datetime.strftime(datetime.now(), '%Y-%m-%d')+'.csv')
print('Сохранено как hh_ru_'+startDate+'-'+datetime.strftime(datetime.now(), '%Y-%m-%d')+'.csv')

Сохранено как hh_ru_2024-06-23-2024-07-18.csv
